In [ ]:
import cv2
import numpy as np
import threading
import datetime
from playsound import playsound
from twilio.rest import Client
import os
import logging

# Create a directory for logs and screenshots
output_dir = "motion_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set up logging
log_file = os.path.join(output_dir, "motion_detection.log")
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Twilio credentials
account_sid = 'twillio_id'
auth_token = 'your_auth-id'
twilio_whatsapp_from = 'whatsapp:+twilio no.'
twilio_whatsapp_to = 'whatsapp:+your no.'

# Font for text display
font = cv2.FONT_HERSHEY_SIMPLEX

def calculate_frame_difference(f1, f2):
    f1_32 = np.float32(f1)
    f2_32 = np.float32(f2)
    diff32 = f1_32 - f2_32
    norm32 = np.sqrt(diff32[:, :, 0]**2 + diff32[:, :, 1]**2 + diff32[:, :, 2]**2) / np.sqrt(255**2 + 255**2 + 255**2)
    dist = np.uint8(norm32 * 255)
    return dist

def play_sound():
    try:
        playsound('motion_detected_sound.wav')
    except Exception as e:
        logging.error(f"Error playing sound: {e}")

def send_whatsapp_message():
    client = Client(account_sid, auth_token)
    try:
        message = client.messages.create(
            from_=twilio_whatsapp_from,
            to=twilio_whatsapp_to,
            body=f"Motion detected at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}!"
        )
        logging.info(f"WhatsApp message sent successfully! SID: {message.sid}")
    except Exception as e:
        logging.error(f"Error sending WhatsApp message: {e}")

def save_screenshot_with_circle(frame, contours):
    try:
        x, y, w, h = cv2.boundingRect(contours)
        center = (x + w // 2, y + h // 2)
        radius = max(w, h) // 2

        screenshot_frame = frame.copy()
        cv2.circle(screenshot_frame, center, radius, (0, 255, 0), 3)

        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = os.path.join(output_dir, f"motion_screenshot_{timestamp}.png")
        cv2.imwrite(filename, screenshot_frame)
        logging.info(f"Screenshot saved: {filename}")
    except Exception as e:
        logging.error(f"Error saving screenshot: {e}")

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    logging.error("Error: Could not open webcam.")
    exit()

_, f1 = cap.read()
_, f2 = cap.read()

motion_threshold = 5000
motion_detected = False
screenshot_taken = False
blink_state = False
sound_thread = None

while True:
    ret, f3 = cap.read()
    if not ret:
        break

    dist = calculate_frame_difference(f1, f3)
    f1 = f2
    f2 = f3

    mod = cv2.GaussianBlur(dist, (9, 9), 0)
    _, thresh = cv2.threshold(mod, 50, 255, 0)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    motion_area = np.count_nonzero(thresh)

    if motion_area > motion_threshold:
        if not motion_detected:
            logging.info("Motion detected!")
            motion_detected = True

            if sound_thread is None or not sound_thread.is_alive():
                sound_thread = threading.Thread(target=play_sound)
                sound_thread.start()

            send_whatsapp_message()

            if not screenshot_taken and contours:
                save_screenshot_with_circle(f3, max(contours, key=cv2.contourArea))
                screenshot_taken = True

    else:
        motion_detected = False
        screenshot_taken = False

    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cv2.putText(f3, current_time, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

    rows, cols, _ = f3.shape
    frame_resized = cv2.resize(f3, (int(cols / 3), int(rows / 3)))
    mod_resized = cv2.resize(mod, (int(cols / 3), int(rows / 3)))
    thresh_resized = cv2.resize(thresh, (int(cols / 3), int(rows / 3)))

    combined_frame = np.hstack((
        frame_resized,
        cv2.cvtColor(mod_resized, cv2.COLOR_GRAY2BGR),
        cv2.cvtColor(thresh_resized, cv2.COLOR_GRAY2BGR)
    ))

    sd_region = np.zeros((50, combined_frame.shape[1], 3), dtype=np.uint8)
    status_text = "Motion Detected!" if motion_detected else "No Motion"
    status_color = (0, 0, 255) if motion_detected else (0, 255, 0)
    cv2.putText(sd_region, status_text, (10, 35), font, 1, status_color, 2, cv2.LINE_AA)

    final_display = np.vstack((combined_frame, sd_region))

    # Resize and display the window
    cv2.namedWindow('All Windows', cv2.WINDOW_NORMAL)  # Enable resizing
    cv2.resizeWindow('All Windows', 1920, 1080)  # Set to large window size
    cv2.imshow('All Windows', final_display)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
